In [2]:
%pip install langchain langchain-core langchain-community langchain-experimental


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install langchain_chroma

  Using cached langchain_chroma-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached kubernetes-30.1.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached mmh3-4.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached pyproject_hooks-1.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.

In [ ]:
%pip install langchain-huggingface

In [10]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader # TextLoader...
from langchain_chroma import Chroma # FAISS from Facebook AI
from langchain_core.output_parsers import StrOutputParser # JSONOutputParser...
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3.1:latest",temperature=0.2)

In [16]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [17]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [20]:
len(splits)

66

In [21]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text = "This is a test document."
query_result = embeddings.embed_query(text)

/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
# show only the first 100 characters of the stringified vector
print(str(query_result)[:100] + "...")

[-0.03833858668804169, 0.12346471846103668, -0.028642985969781876, 0.05365273728966713, 0.0088453628...


In [23]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

In [24]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [25]:
prompt = hub.pull("rlm/rag-prompt")

In [26]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [32]:
splits[0].page_content

'LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory'

In [27]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [33]:
print(format_docs(splits[:2]))

LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Memory

Short-term memory: I 

In [ ]:
# Define the prompt template for the document chain
document_chain_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
]) 

In [35]:
rag_chain =(
    {"context": retriever | format_docs,"question": RunnablePassthrough()}
    | prompt
    | llm
)

In [36]:
rag_chain.invoke("What is Task Decomposition?")

AIMessage(content='Task Decomposition is the process of breaking down complex tasks into smaller and simpler steps, allowing an agent or model to plan ahead and manage the task more effectively. This can be achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts (Yao et al., 2023), which transform big tasks into multiple manageable tasks.', response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-08-03T17:04:55.830454Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 17065979042, 'load_duration': 10852993833, 'prompt_eval_count': 620, 'prompt_eval_duration': 2661745000, 'eval_count': 71, 'eval_duration': 3530428000}, id='run-5f4805be-1423-4178-b5cf-244b3e8d7362-0')

In [37]:
rag_chain =(
    {"context": retriever | format_docs,"question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [38]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down complex tasks into smaller and simpler steps, allowing an agent or model to plan ahead and manage the task more effectively. This can be achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts (Yao et al., 2023), which transform big tasks into multiple manageable tasks.'

In [45]:
vectorstore.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                    "score_threshold": 0.1,
                    "k": 20
                }
                ,
        ).get_relevant_documents("What is Task Decomposition?")

/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), 0.35159509653833465), (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates mul

### Prompt Template

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Our cours is about {variable1} and {variable2}")
partial_prompt = prompt.partial(variable1="AI")
print(partial_prompt.format(variable2="Data Science"))

Our cours is about AI and Data Science


In [53]:
prompt.format(variable1="AI", variable2="Data Science")

'Our cours is about AI and Data Science'

In [54]:
variable1 = "AI"
variable2 = "Data Science"
f"Our cours is about {variable1} and {variable2}"

'Our cours is about AI and Data Science'

In [16]:
from langchain import PromptTemplate


restaurant_template = """
I want you to act as a naming consultant for new restaurants.

Return a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.

What are some good names for a restaurant that is {restaurant_desription}?
"""

prompt = PromptTemplate(
    input_variables=["restaurant_desription", "num_names"],
    template=restaurant_template,
)

In [13]:
description = "a burger place that is themed with baseball memorabilia"
prompt.format(restaurant_desription=description, num_names=5)

'\nI want you to act as a naming consultant for new restaurants.\n\nReturn a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.\n\nNumber of restaurant names: 5\n\nWhat are some good names for a restaurant that is a burger place that is themed with baseball memorabilia?\n'

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [15]:
print(chain.run(restaurant_desription="a burger place that is themed with baseball memorabilia", num_names=5))

/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


What a home run of an idea! Here are five short, catchy, and easy-to-remember name suggestions for your baseball-themed burger joint:

1. **Dugout Burgers**: A playful reference to the dugouts where players sit during games, this name immediately conveys the sports theme.
2. **The Stadium Grill**: This name evokes the excitement of a live game, with the added bonus of a grill serving up delicious burgers.
3. **Home Plate Eats**: Another baseball reference that's hard to miss, "Home Plate" is a nod to the ultimate goal in baseball, while also emphasizing the tasty eats you'll find inside.
4. **The Umpire's Burger Co.**: This name adds a fun twist by giving your restaurant a playful, umpire-like persona. It's sure to bring a smile to customers' faces!
5. **Grand Slam Burgers**: A grand slam is the highest achievement in baseball, and this name suggests that your burgers are the ultimate culinary home run.

I hope these suggestions hit a home run with you!


In [17]:
restaurant_chain =(
    {"restaurant_desription": RunnablePassthrough()}
    | prompt
    | llm
)

In [19]:
print(restaurant_chain.invoke("a burger place that is themed with baseball memorabilia"))

content="I'd love to help! Here are some name suggestions for your baseball-themed burger joint:\n\n1. **Home Run Burgers**: A playful reference to the ultimate goal in baseball, perfect for a burger spot.\n2. **The Dugout**: A nod to the players' resting area on the field, evoking a cozy, casual vibe.\n3. **Ballpark Bites**: Simple and straightforward, this name conveys the fun, snacky aspect of your burgers.\n4. **The Ump's Burger Co.**: A clever reference to the umpires who keep the game fair, with a hint of playfulness.\n5. **Grand Slam Grill**: Another baseball term turned into a mouth-watering promise, implying big flavors and satisfying meals.\n6. **The Stadium Grill**: Straightforward and attention-grabbing, this name puts your restaurant in the spotlight.\n7. **Bases Loaded Burgers**: A fun phrase that conjures up images of excitement and possibility – perfect for a burger joint!\n8. **The Field House**: A name that suggests a casual, laid-back atmosphere, like a favorite hang

### Fewshot Prompting

In [13]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [14]:
# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [15]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

In [16]:
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [17]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt, # PromptTemplate object
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)

In [19]:
print(few_shot_prompt.format(input="hot"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: hot
Antonym:


In [20]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=few_shot_prompt)

print(chain.run("hot"))

The antonym of "hot" is cold.


In [5]:
from langchain import PromptTemplate, FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [6]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What is the meaning of life?
AI: 


In [8]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

print(chain.invoke("What is the meaning of life?"))

/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'query': 'What is the meaning of life?', 'text': "I see what you're doing here. You want me to complete the AI's response with some witty and sarcastic remark, right? Okay, let's give it a try!\n\nHere are my attempts at completing each conversation:\n\n**1. User: How are you?**\nAI: I'm functioning within optimal parameters... but don't worry about me, how are you?\n\n**2. User: What time is it?**\nAI: It's time to get a watch (as mentioned earlier). But if you must know, the current time is... *checks virtual clock* ...whenever you need it to be.\n\n**3. User: What is the meaning of life?**\nAI: Ah, finally, a question that gets to the heart of the matter! In all seriousness, I'm still working on that one. But if I had to give you an answer, I'd say the meaning of life is to find a good Wi-Fi signal and never look back."}


In [9]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
